# 1. Getting the data

- link to dataset: [https://www.kaggle.com/datasets/aslanahmedov/predict-term-deposit](https://www.kaggle.com/datasets/aslanahmedov/predict-term-deposit)

In [1]:
data = 'https://www.kaggle.com/datasets/aslanahmedov/predict-term-deposit/download?datasetVersionNumber=1'

In [3]:
!wget $data

--2022-10-27 18:19:42--  https://www.kaggle.com/datasets/aslanahmedov/predict-term-deposit/download?datasetVersionNumber=1
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Faslanahmedov%2Fpredict-term-deposit%2Fversions%2F1%3Fresource%3Ddownload [following]
--2022-10-27 18:19:42--  https://www.kaggle.com/account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Faslanahmedov%2Fpredict-term-deposit%2Fversions%2F1%3Fresource%3Ddownload
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘download?datasetVersionNumber=1’

download?datasetVer     [ <=>                ]   6.4